# Inspection de datasets : analyse de fichiers NIfTI

Ce chapitre explore comment explorer et analyser des fichiers NIfTI (Neuroimaging Informatics Technology Initiative), un format courant en imagerie médicale et neuroscience. Les fichiers NIfTI peuvent contenir des images 3D, 4D ou plus.

---

## Concepts clés

- **Format NIfTI** : conteneur standard pour les images volumétriques
- **Affine matrix** : transformation spatiale en coordonnées réelles
- **Dimensions et voxels** : structure des données 3D
- **Inspection récursive** : naviguer dans les dossiers pour découvrir les données
- **Visualisation multi-plan** : afficher des coupes selon différents axes

In [ ]:
# Importation des bibliothèques
import os
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib

# Importer les fonctions utilitaires personnalisées
from utils import split_name_with_nii, get_slice, get_nifti_data, get_nifti_header_info, summarize_intensities

# Configurer NumPy pour afficher les nombres sans notation scientifique
np.set_printoptions(suppress=True)

---

## Navigation récursive et inspection des fichiers NIfTI

La fonction `os.walk()` permet de naviguer récursivement dans une arborescence de dossiers pour découvrir tous les fichiers NIfTI. Pour chaque fichier, on extrait les métadonnées et on visualise des coupes.

In [ ]:
# Parcourir tous les fichiers du répertoire courant et ses sous-dossiers
# topdown=False explore les dossiers enfants d'abord
for root, dirs, files in os.walk("data/", topdown=False):
    for filename in files:
        filepath = os.path.join(root, filename)
        base, ext = split_name_with_nii(filepath)
        
        # Traiter seulement les fichiers NIfTI
        if ext in ['.nii', '.nii.gz']:
            print(f"\n{'='*50}")
            print(f"Fichier : {filepath}")
            
            # Charger le fichier NIfTI
            img_nifti = nib.load(filepath)
            img_data = get_nifti_data(img_nifti)
            
            # Si l'image est 4D (par exemple, données de diffusion),
            # prendre seulement le premier volume
            if img_data.ndim > 3:
                last_dim = img_data.shape[-1]
                img_data = img_data[..., 0]
            else:
                last_dim = 0
            
            # Extraire les informations du header NIfTI
            affine_matrix, dimensions, voxel_sizes, voxel_order = \
                get_nifti_header_info(img_nifti)
            
            # Créer une figure avec 3 coupes (axial, coronal, sagittal)
            fig, axs = plt.subplots(1, 3, figsize=(15, 5))
            
            # Afficher une coupe pour chaque orientation
            for i, axis_name in enumerate(['axial', 'coronal', 'sagittal']):
                slice_data = get_slice(img_data, axis_name)
                axs[i].imshow(slice_data, cmap='gray')
                axs[i].set_title(f"Coupe {axis_name}")
                axs[i].axis('off')
            
            fig.tight_layout()
            plt.show()
            
            # Afficher les métadonnées
            print(f"\nMatrice affine :\n{np.round(affine_matrix, 3)}")
            print(f"Dimensions (voxels) : {dimensions}")
            print(f"Taille des voxels (mm) : {np.round(voxel_sizes, 3)}")
            print(f"Ordre des axes : {voxel_order}")
            print(f"Type de données : {img_data.dtype}")
            print(f"Statistiques des intensités :\n{summarize_intensities(img_data)}")
            
            # Afficher si c'est une image 4D
            if last_dim:
                print(f"Dimension 4D (nombre de volumes) : {last_dim}")
            
            print(f"{'='*50}")

---

## Résumé

- Les fichiers **NIfTI** contiennent des images volumétriques 3D ou 4D
- La **matrice affine** définit la transformation spatiale en coordonnées réelles
- Les **voxels** ont des tailles (résolutions) différentes selon les axes
- La **navigation récursive** permet de découvrir automatiquement tous les fichiers
- La **visualisation multi-plan** révèle la structure complète des données

Prochain chapitre : `h_cell_counting.ipynb`